# December 19th

Yay meta programming!

In [27]:
example = [
    "px{a<2006:qkq,m>2090:A,rfg}",
    "pv{a>1716:R,A}",
    "lnx{m>1548:A,A}",
    "rfg{s<537:gd,x>2440:R,A}",
    "qs{s>3448:A,lnx}",
    "qkq{x<1416:A,crn}",
    "crn{x>2662:A,R}",
    "in{s<1351:px,qqz}",
    "qqz{s>2770:qs,m<1801:hdj,R}",
    "gd{a>3333:R,R}",
    "hdj{m>838:A,pv}",
    "",
    "{x=787,m=2655,a=1222,s=2876}",
    "{x=1679,m=44,a=2067,s=496}",
    "{x=2036,m=264,a=79,s=2244}",
    "{x=2461,m=1339,a=466,s=291}",
    "{x=2127,m=1623,a=2188,s=1013}"
]

examplepartonesoution = 19114

19114

In [28]:
function createworkflow(workflow)
    steps = split(workflow, ',')
    if length(steps) == 1
        return () -> steps[1]
    end
    fn = "function (x,m,a,s)\n"
    for step in steps
        parts = split(step, ':')
        if length(parts) == 2
            fn = fn * "if $(parts[1]) \n \"$(parts[2])\"\n else"
        else
            fn = fn * "\n \"$(parts[1])\" \n end \n"
        end
    end
    fn = fn * "end"
    return eval(Meta.parse(fn))
end

function createworkflows(workflows)
    allworkflows = Dict()
    for workflow in workflows
        idx = findfirst('{', workflow)
        key = workflow[begin:idx-1]
        fn = createworkflow(workflow[idx+1:end-1])
        allworkflows[key] = fn
    end
    return allworkflows
end

function executeworkflows(workflows, x, m, a, s)
    next = "in"
    while next != "A" && next != "R"
        wf = workflows[next]
        next = Base.invokelatest(wf, x, m, a, s)
    end
    return next == "A"
end

wf = createworkflow("a<2006:qkq,m>2090:A,rfg")
wf(0, 1900, 2007, 0)

function getparts(ln)
    (xp, mp, ap, sp) = split(ln[begin+1:end-1], ',')
    (
        parse(Int, xp[begin+2:end]),
        parse(Int, mp[begin+2:end]),
        parse(Int, ap[begin+2:end]),
        parse(Int, sp[begin+2:end])
    )
end


getparts (generic function with 1 method)

Well Part 2 needs a decision tree let's build one.

In [29]:
mutable struct CompNode
    fromnode
    field::Char
    comparison::Char
    target::Int
    then
    orelse
end

mutable struct RefNode
    fromnode
    target::AbstractString
end

mutable struct AcceptRejectNode
    fromnode
    accept::Bool
end

function createworkflownode(ln)
    parts = split(ln, ',')
    p = parts[end] == "A" || parts[end] == "R" ? AcceptRejectNode(nothing, parts[end] == "A") : RefNode(nothing, parts[end])
    for partidx in range(lastindex(parts) - 1, firstindex(parts), step=-1)
        (comp, target) = split(parts[partidx], ':')
        then = target == "A" || target == "R" ? AcceptRejectNode(nothing, target == "A") : RefNode(nothing, target)
        next = CompNode(
            nothing,
            comp[begin],
            comp[begin+1],
            parse(Int, comp[begin+2:end]),
            then,
            p
        )
        p.fromnode = next
        then.fromnode = next
        p = next
    end
    return p
end

function createworkflownodes(workflows)
    allworkflows = Dict()
    for workflow in workflows
        idx = findfirst('{', workflow)
        key = workflow[begin:idx-1]
        n = createworkflownode(workflow[idx+1:end-1])
        allworkflows[key] = n
    end
    return allworkflows
end

function dereferencenodes(allnodes, parent, node)
    if typeof(node) == AcceptRejectNode
        AcceptRejectNode(parent, node.accept)
    elseif typeof(node) == RefNode
        dereferencenodes(allnodes, parent, allnodes[node.target])
    else
        then = dereferencenodes(allnodes, nothing, node.then)
        orelse = dereferencenodes(allnodes, nothing, node.orelse)
        n = CompNode(
            parent,
            node.field,
            node.comparison,
            node.target,
            then,
            orelse
        )
        then.fromnode = n
        orelse.fromnode = n
        n
    end
end

function executenodes(node, x, m, a, s)
    if typeof(node) == AcceptRejectNode
        return node.accept
    end
    v1 = if node.field == 'x'
        x
    elseif node.field == 'm'
        m
    elseif node.field == 'a'
        a
    elseif node.field == 's'
        s
    end
    n = if node.comparison == '<'
        v1 < node.target
    else
        v1 > node.target
    end
    return executenodes(n ? node.then : node.orelse, x, m, a, s)
end


executenodes (generic function with 1 method)

In [30]:
function partone(contents)
    si = findfirst(l -> l == "", contents)
    workflows = createworkflownodes(contents[begin:si-1])
    startnode = dereferencenodes(workflows, nothing, workflows["in"])
    r = 0
    for part in contents[si+1:end]
        (x, m, a, s) = getparts(part)
        if executenodes(startnode, x, m, a, s)
            r = r + x + m + a + s
        end
    end
    return r
end

partone(example)

19114

## Part 2

We need to find the number of possibilites again we'll lean into recursive functions

In [33]:
function countpossibilites(node, x, m, a, s)
    if typeof(node) == AcceptRejectNode
        if node.accept
            return length(x) * length(m) * length(a) * length(s)
        else
            return 0
        end
    end
    if node.comparison == '<'
        if node.field == 'x'
            return countpossibilites(
                node.then,
                x.start:node.target-1,
                m,
                a,
                s
            ) + countpossibilites(
                node.orelse,
                node.target:x.stop,
                m,
                a,
                s
            )
        elseif node.field == 'm'
            return countpossibilites(
                node.then,
                x,
                m.start:node.target-1,
                a,
                s
            ) + countpossibilites(
                node.orelse,
                x,
                node.target:m.stop,
                a,
                s
            )
        elseif node.field == 'a'
            return countpossibilites(
                node.then,
                x,
                m,
                a.start:node.target-1,
                s
            ) + countpossibilites(
                node.orelse,
                x,
                m,
                node.target:a.stop,
                s
            )
        else
            return countpossibilites(
                node.then,
                x,
                m,
                a,
                s.start:node.target-1,
            ) + countpossibilites(
                node.orelse,
                x,
                m,
                a,
                node.target:s.stop,
            )
        end
    else
        if node.field == 'x'
            return countpossibilites(
                node.then,
                node.target+1:x.stop,
                m,
                a,
                s
            ) + countpossibilites(
                node.orelse,
                x.start:node.target,
                m,
                a,
                s
            )
        elseif node.field == 'm'
            return countpossibilites(
                node.then,
                x,
                node.target+1:m.stop,
                a,
                s
            ) + countpossibilites(
                node.orelse,
                x,
                m.start:node.target,
                a,
                s
            )
        elseif node.field == 'a'
            return countpossibilites(
                node.then,
                x,
                m,
                node.target+1:a.stop,
                s
            ) + countpossibilites(
                node.orelse,
                x,
                m,
                a.start:node.target,
                s
            )
        else
            return countpossibilites(
                node.then,
                x,
                m,
                a,
                node.target+1:s.stop
            ) + countpossibilites(
                node.orelse,
                x,
                m,
                a,
                s.start:node.target
            )
        end
    end
end

function parttwo(contents)
    si = findfirst(l -> l == "", contents)
    workflows = createworkflownodes(contents[begin:si-1])
    startnode = dereferencenodes(workflows, nothing, workflows["in"])
    countpossibilites(startnode, 1:4000, 1:4000, 1:4000, 1:4000)
end

parttwo(example) == 167409079868000

true

## Results

In [34]:
include("./aoc.jl")

execute(19, partone, parttwo; keepempty=true)

480738
131550418841958
